# 1. DeBug

## 反汇编器`dis`

使用内置的dis模块，查看函数的反汇编代码：

In [2]:
import dis

def add(x, y):
    return x + y + 11*24

dis.dis(add)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 LOAD_CONST               1 (264)
              8 BINARY_ADD
             10 RETURN_VALUE


## 性能测试`timeit`
性能测试，可以直接使用`timeit`模块

In [8]:
import timeit

def foo():
    add(1,2)

# 默认执行100万次
timeit.timeit(setup='from __main__ import foo',stmt = 'foo()', number=1000000)

0.13255862500000148

圈复杂度：代表程序内线性独立路径的数量。值越大表示执行路径越多，逻辑越复杂。

todo

# 2. 模块

## 2.1 装饰器

`@wrapt.decorator`函数实现的装饰器，有一个知名的问题。那就是如果装饰类里的方法，因为类方法默认第一个参数是`self`，会出现问题。此时可以使用此第三方库的，解决此问题。

`@functools.wraps(func)`装饰函数，避免元数据覆盖。`functools.update_wrapper(self, function)`类专时期避免元数据覆盖

`@lru_catche(max_size=None)`给函数增加缓存功能：如果某个函数的返回值相对于输入值是固定的，那么可以使用缓存功能，避免重复计算,max_size表示最多保存多少缓存结果。默认为128。


`@property` 将类的方法转化成属性，并可以自定义属性的get set del；

`@abstractmethod` 装饰类方法后，继承此类的类，必须实现被装饰的类方法



## 2.2 functools

`functools.reduce()` : reduce() 函数的第一个参数是接受两个参数的函数，第二个参数是一个可迭代的对象。假如有个接受两个参数的 $fn$ 函数和一个 lst 列表。调用 reduce(fn, lst) 时，$fn$ 会应用到第一对元素上，即 $fn(lst[0], lst[1])$，生成第一个结果 $r1$。然后，$fn$ 会应用到 $r1$ 和下一个元素 上，即$fn(r1, lst[2])$，生成第二个结果$r2$。接着，调用 $fn(r2, lst[3])$，生成 $r3$……直到最后一个元素，返回最后得到的结果 $rN$。

In [9]:
import functools as func

func.reduce(lambda a, b: a+b, range(1,11))

55

## 2.3 魔术方法

`__init_subclass__` 是类的一个特殊钩子方法，当类在派生出子类时，会触发此方法。